# Lab 3: Titanic

### Description

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew (32% survival rate). This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

### Overview

The data has been split into two groups:

training set (titanic_train.csv)
test set (titanic_test.csv)

The training set should be used to build your machine learning models. The training set includes the target (outcome) value for each passenger. Your model will be based on “features” like the passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. The test set lacks the target variable. It is the task of your model to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

### Data Dictionary

Survived: 0 = No, 1 = Yes 
Pclass: class 1 = 1st, 2 = 2nd, 3 = 3rd 
Sex: gender
Age: Age in years 
SibSp: # of siblings / spouses aboard the Titanic 
Parch: # of parents / children aboard the Titanic 
Ticket: ticket number
Fare: Passenger fare 
Cabin: Cabin number 
Embarked: Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton


   **Variable Notes**
   
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, it is in the form of xx.5

SibSp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

Parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore Parch=0 for them.

In [4]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("titanic_train.csv")
df.head()

### Distribution of numerical features

In [ ]:
df.describe()

In [ ]:
df.info()

### Distribution of categorical features

In [ ]:
df.describe(include=["O"])

Which features can be dropped?
Which features may we want to complete/impute?